#1. Data

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

!pip install -q tensorflow-hub # 최신버젼 데이터 셋
!pip install -q tfds-nightly # 미리 훈련된 모델을 쉽게 사용할 수 있도록 도와주는 라이브러리
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print(tf.__version__)
print(hub.__version__)
print('GPU', 'On' if tf.config.experimental.list_physical_devices("GPU") else 'Off')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 19.4 MB/s eta 0:00:00
2.15.0
0.15.0
GPU On


In [2]:
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True  # 리뷰테스트와 감성점수 둘다 제공
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHCDHFE/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHCDHFE/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHCDHFE/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
len(train_data), len(validation_data), len(test_data)

(15000, 10000, 25000)

In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [5]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

#2. Model

In [6]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:5])

<tf.Tensor: shape=(5, 20), dtype=float32, numpy=
array([[ 1.7657859 , -3.882232  ,  3.913424  , -1.5557289 , -3.3362343 ,
        -1.7357956 , -1.9954445 ,  1.298955  ,  5.081597  , -1.1041285 ,
        -2.0503852 , -0.7267516 , -0.6567596 ,  0.24436145, -3.7208388 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489715 , -1.1315986 ],
       [ 1.8804485 , -2.5852385 ,  3.4066994 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.7855542 ,  1.3874227 ,  3.8476458 , -0.9256539 ,
        -1.896706  ,  1.2113281 ,  0.11474716,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015903 , -1.6390051 ],
       [ 0.71152216, -0.63532174,  1.7385626 , -1.1168287 , -0.54515934,
        -1.1808155 ,  0.09504453,  1.4653089 ,  0.66059506,  0.79308075,
        -2.2268343 ,  0.07446616, -1.4075902 , -0.706454  , -1.907037  ,
         1.4419788 ,  1.9551864 , -0.42660046, -2.8022065 ,  0.43727067],
       [ 1.5164999 , -0.7103405 ,  1.8556769 , -1.2033532 , -1.3105817 ,

In [7]:
model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [9]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512))

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


30/30 [==============================] - 15s 395ms/step - loss: 0.9776 - accuracy: 0.4975 - val_loss: 0.7294 - val_accuracy: 0.5176
Epoch 2/20
30/30 [==============================] - 6s 187ms/step - loss: 0.6910 - accuracy: 0.5666 - val_loss: 0.6629 - val_accuracy: 0.6046
Epoch 3/20
30/30 [==============================] - 7s 203ms/step - loss: 0.6483 - accuracy: 0.6277 - val_loss: 0.6316 - val_accuracy: 0.6439
Epoch 4/20
30/30 [==============================] - 7s 223ms/step - loss: 0.6145 - accuracy: 0.6675 - val_loss: 0.6011 - val_accuracy: 0.6819
Epoch 5/20
30/30 [==============================] - 5s 176ms/step - loss: 0.5814 - accuracy: 0.7065 - val_loss: 0.5719 - val_accuracy: 0.7096
Epoch 6/20
30/30 [==============================] - 7s 212ms/step - loss: 0.5485 - accuracy: 0.7366 - val_loss: 0.5436 - val_accuracy: 0.7355
Epoch 7/20
30/30 [==============================] - 8s 256ms/step - loss: 0.5154 - accuracy: 0.7633 - val_loss: 0.5138 - val_accuracy: 0.7644
Epoch 8/20
30/30

In [10]:
results = model.evaluate(test_data.batch(512))

for metrics, value in zip(model.metrics_names, results):
    print("Metrics: %s Value: %.3f" % (metrics, value))

49/49 [==============================] - 2s 35ms/step - loss: 0.3203 - accuracy: 0.8632
Metrics: loss Value: 0.320
Metrics: accuracy Value: 0.863
